In [1]:
import pandas as pd

In [2]:
mock = "ec_22_09_2023_31_08_2024.csv"

In [3]:
df = pd.read_csv(mock, encoding= 'unicode_escape')

In [4]:
for col in df.columns:
    print (col)

Data operazione
Data contabile
Tipologia
Nome
Descrizione
Importo ( âÂ )


need to clean the name of the Importo column

In [5]:
df["Importo"] = df[list(df.columns)[-1]]
del df[list(df.columns)[-2]]
for col in df.columns:
    print (col)

Data operazione
Data contabile
Tipologia
Nome
Descrizione
Importo


we are only interested in the column tipologia

In [6]:
df["Tipologia"]

0                 Risparmi
1                   PagoPA
2                   PagoPA
3       Bonifico ordinario
4                Pagamento
               ...        
1071             Pagamento
1072             Pagamento
1073              Risparmi
1074             Pagamento
1075             Pagamento
Name: Tipologia, Length: 1076, dtype: object

as we can see some values are nan

In [7]:
# commented for privacy
# df[df['Tipologia'].isna()]

checking in the csv file those rows are badly parsed

everything is in the first column

need to clean the name of the Importo column

In [8]:
# commented for privacy
# list(df[df['Tipologia'].isna()]["Data operazione"])

as we can see those are either wire transfers or alike operations

even tho they are all in the same column they still are separated by a ,

so we can just clean the initial file and read as usual

checking the csv file it's clear that we just need to remove the " chars from the affected rows

after further investigations it came to mind that the fields Nome and Descrizione might contain " chars, the following implementation

as one can see in the code below i split the row by the "," and afterward i clean it

by analysis it appeared that the number of elements in irregular rows variates (from the original 6) between 7 and 8

since the first element is always the "data operazione", the second is always "data contabile", the third is "tipologia" and the LAST is "importo" i decided to split the values based on the number of total elements

if the total number of elements is 7 the middle value goes to the left field

if the total number of elements is 8 one middle value goes to the left and the other to the right

In [9]:
import os

def clean_csv(output_file):
    with open(output_file, 'w', newline='\n') as outfile:
        write_header(outfile, df.columns)
        for row in df.iterrows():
            row_data = process_row(row[1])
            outfile.write(row_data + "\n")

def write_header(outfile, columns):
    header = ",".join(columns) + "\n"
    outfile.write(header)

def process_row(row):
    row_list = [str(col) for col in row]
    if all(value == "nan" for value in row_list[1:6]):
        return clean_splitted_row(row_list[0])
    return ",".join(row_list).replace('"', '')

def clean_splitted_row(row_string):
    splitted_row = row_string.split(",")
    if len(splitted_row) <= 6:
        return row_string.replace('"', '')

    splitted_row = [s.replace('"', '') for s in splitted_row]
    data_operazione, data_contabile, tipologia = splitted_row[:3]
    importo = splitted_row[-1]
    
    if len(splitted_row) == 7:
        nome = splitted_row[3] + splitted_row[4]
        descrizione = splitted_row[5]
    else:
        nome = splitted_row[3] + splitted_row[4]
        descrizione = splitted_row[5] + splitted_row[6]
    
    cleaned_row = [data_operazione, data_contabile, tipologia, nome, descrizione, importo]
    return ",".join(cleaned_row).replace('"', '')

os.makedirs('generatedFiles', exist_ok=True)
output_file = 'generatedFiles/cleaned_csv.csv'
clean_csv(output_file)


now we can read the dataframe without particular issues